# OPENAI-NOTEBOOK4  AUTOMATIC RECIPE CREATOR

From course by Jose Portilla

Code creates an automatic recipe creator using ChatGPT and images from DALLE

In [43]:
import os
import openai
import re
import requests
import shutil

In [5]:
#os.environ['OPEN_AI_KEY'] = ''
openai.api_key = os.getenv('OPEN_AI_KEY')

## Generation of Text Using Completion API

In [25]:
# Create the prompt

def create_dish_prompt(ingredients):
    prompt = f"Create a detailed recipe using only the following ingredients: {', '.join(ingredients)}.\n"\
            +f"Additionally, assign a title starting with 'Recipe Title: ' to this recipe."
    return prompt

In [26]:
# Test the prompt

print(create_dish_prompt(['cheese','mushroom','bacon']))

Create a detailed recipe using only the following ingredients: cheese, mushroom, bacon.
Additionally, assign a title starting with 'Recipe Title: ' to this recipe.


In [27]:
# Run function
ing = ['cheese', 'mushroom', 'bacon']
prompt = create_dish_prompt(ing)
prompt

"Create a detailed recipe using only the following ingredients: cheese, mushroom, bacon.\nAdditionally, assign a title starting with 'Recipe Title: ' to this recipe."

In [28]:
# Create the response
response = openai.Completion.create(engine = 'text-davinci-003', 
                                   prompt = prompt,
                                   max_tokens = 512,
                                   temperature = 0.7)

In [29]:
# Print the recipe
print(response['choices'][0]['text'])



Recipe Title: Cheesy Mushroom Bacon Skillet

Ingredients:
- 4 slices of bacon, cut into small pieces
- 1/2 cup of mushrooms, sliced
- 1/2 cup of shredded cheese (cheddar, mozzarella, etc.)

Instructions:
1. In a skillet over medium-high heat, cook bacon pieces until crisp, stirring occasionally.
2. Once bacon is cooked, add mushrooms and stir to combine. Cook until mushrooms are tender, about 5 minutes.
3. Reduce heat to low and add shredded cheese. Stir until cheese is melted and everything is combined.
4. Serve hot. Enjoy!


In [32]:
recipe = response['choices'][0]['text']
recipe

'\n\nRecipe Title: Cheesy Mushroom Bacon Skillet\n\nIngredients:\n- 4 slices of bacon, cut into small pieces\n- 1/2 cup of mushrooms, sliced\n- 1/2 cup of shredded cheese (cheddar, mozzarella, etc.)\n\nInstructions:\n1. In a skillet over medium-high heat, cook bacon pieces until crisp, stirring occasionally.\n2. Once bacon is cooked, add mushrooms and stir to combine. Cook until mushrooms are tender, about 5 minutes.\n3. Reduce heat to low and add shredded cheese. Stir until cheese is melted and everything is combined.\n4. Serve hot. Enjoy!'

In [40]:
# Clean up the result to extract the title to send to DALLE

def extract_title(prompt):
    return re.findall('^.*Recipe Title: .*$', prompt, re.MULTILINE)[0].strip().split('Recipe Title: ')[-1]

In [41]:
extract_title(recipe)

'Cheesy Mushroom Bacon Skillet'

In [51]:
recipe_title = extract_title(recipe)

In [52]:
# Trial a second prompt with style images

def dalle2_prompt(recipe_title):
    prompt_dalle2 = f"{recipe_title}, professional food photography, 15mm, studio lighting"
    return prompt_dalle2

dalle2_prompt(recipe_title)

'Cheesy Mushroom Bacon Skillet, professional food photography, 15mm, studio lighting'

## Send to DALLE

* Can request 256, 512 or 1024 pixels for images (larger images cost more)
* Can ask for up to 10 images back from the Image API call
* Can use images as a prompt as well as text - open("pic.png", 'rb')

In [53]:
# Obtain the image
image_response = openai.Image.create(prompt = dalle2_prompt(recipe_title),
                                    n = 1,
                                    size = '512x512'
                                    )

In [54]:
image_url = image_response['data'][0]['url']

In [55]:
# Save the image

def save_image(image_url, file_name):
    image_res = requests.get(image_url, stream = True)
    if image_res.status_code == 200: # check ok to download
        with open(file_name, 'wb') as f:
            shutil.copyfileobj(image_res.raw, f) # image as raw info
    else:
        print('ERROR LOADING IMAGE')
        
    return image_res.status_code
            

In [56]:
save_image(image_url, 'example_download.png')

200